In [2]:
import os
import sys
sys.path.append(os.path.abspath("src"))


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from models.module.Inception import Inception
from models.GoogLeNet import GoogLeNet  # GoogLeNet 모델 불러오기



In [11]:
# CUDA 설정

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 20
batch_size = 128
learning_rate = 0.001
num_classes = 10
print(device)

cuda


In [5]:
# CIFAR- 10 데이터셋 로딩 및 전처리
transforms = transforms.Compose([
    transforms.Resize((224, 224)), #GoogLeNet 입력 크기에 맞춤
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5,0.5,0.5))
])

train_datset = datasets.CIFAR10(root = './data', train=True, transform=transforms, download=True)
train_loader = DataLoader(train_datset, batch_size=batch_size, shuffle=True)

100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data


In [12]:
# 모델 및 손실함수, 옵티마이저 초기화
model = GoogLeNet(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr= learning_rate)

In [13]:
def train(model, train_loader, criterion, optimizer, num_epochs):
    model.train()  # 학습 모드 활성화

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        
        print(f"Epoch [{epoch + 1}/{num_epochs}] 시작")  # 에포크 시작 알림

        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            # 옵티마이저 초기화
            optimizer.zero_grad()

            # 모델에 입력을 넣어 결과 얻기
            outputs, aux1, aux2 = model(inputs)
            
            # 손실 계산 (메인 분류기와 두 개의 보조 분류기의 손실을 모두 사용)
            loss1 = criterion(outputs, labels)
            loss2 = criterion(aux1, labels)
            loss3 = criterion(aux2, labels)
            loss = loss1 + 0.3 * loss2 + 0.3 * loss3  # 메인과 보조 분류기 손실 가중치

            # 역전파 및 옵티마이저 업데이트
            loss.backward()
            optimizer.step()

            # 손실 및 정확도 기록
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            # 배치 진행 상태 출력
            print(f"Batch [{batch_idx + 1}/{len(train_loader)}] - Loss: {loss.item():.4f}")
        
        # 에포크당 평균 손실과 정확도 출력
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")
        print("=" * 50)  # 에포크 구분선

    print("학습 완료")


In [14]:
from torchsummary import summary
model = GoogLeNet(num_classes=num_classes).to(device)
summary(model, input_size=(3, 224, 224))  # GoogLeNet의 입력 크기를 지정

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,472
         MaxPool2d-2           [-1, 64, 56, 56]               0
            Conv2d-3           [-1, 64, 56, 56]           4,160
            Conv2d-4          [-1, 192, 56, 56]         110,784
         MaxPool2d-5          [-1, 192, 28, 28]               0
            Conv2d-6           [-1, 64, 28, 28]          12,352
            Conv2d-7           [-1, 96, 28, 28]          18,528
            Conv2d-8          [-1, 128, 28, 28]         110,720
            Conv2d-9           [-1, 16, 28, 28]           3,088
           Conv2d-10           [-1, 32, 28, 28]          12,832
        MaxPool2d-11          [-1, 192, 28, 28]               0
           Conv2d-12           [-1, 32, 28, 28]           6,176
        Inception-13          [-1, 256, 28, 28]               0
           Conv2d-14          [-1, 128,

In [15]:
print(model)

GoogLeNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
  (conv3): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (inception3a): Inception(
    (layer1): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
    (layer2): Sequential(
      (0): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(96, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (layer3): Sequential(
      (0): Conv2d(192, 16, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    )
    (layer4): Sequential(
      (0): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
      (1): Conv2d(192, 32, kernel_size=(1, 1), s

In [1]:
# git test

In [2]:
print("test 할게요")

test 할게요
